In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import json

/Users/swithana/git/d2i/patra-toolkit/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
!pip install tensorflow

SyntaxError: invalid syntax (1648709161.py, line 1)

In [2]:
columns = ["age", "workclass", "fnlwgt", "education", "education-num", "martial-status",
        "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss",
        "hours-per-week", "native-country", "income"]
data = pd.read_csv('/Users/swithana/git/d2i/patra-toolkit/examples/notebooks/data/adult/adult.data', names=columns)

In [3]:
data.head()

,age,workclass,fnlwgt,education,education-num,martial-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


### Pre-processing

In [4]:
label_encoder = LabelEncoder()
data['income'] = label_encoder.fit_transform(data['income'])

# Convert categorical variables to one-hot encoding
data = pd.get_dummies(data, drop_first=True, dtype=float)

data.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,income,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,...,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia
0,39,77516,13,2174,0,40,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,50,83311,13,0,0,13,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,38,215646,9,0,0,40,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,53,234721,7,0,0,40,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,28,338409,13,0,0,40,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
X = data.drop('income', axis=1).values
y = data['income'].values

print(data.columns.tolist())

['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week', 'income', 'workclass_ Federal-gov', 'workclass_ Local-gov', 'workclass_ Never-worked', 'workclass_ Private', 'workclass_ Self-emp-inc', 'workclass_ Self-emp-not-inc', 'workclass_ State-gov', 'workclass_ Without-pay', 'education_ 11th', 'education_ 12th', 'education_ 1st-4th', 'education_ 5th-6th', 'education_ 7th-8th', 'education_ 9th', 'education_ Assoc-acdm', 'education_ Assoc-voc', 'education_ Bachelors', 'education_ Doctorate', 'education_ HS-grad', 'education_ Masters', 'education_ Preschool', 'education_ Prof-school', 'education_ Some-college', 'martial-status_ Married-AF-spouse', 'martial-status_ Married-civ-spouse', 'martial-status_ Married-spouse-absent', 'martial-status_ Never-married', 'martial-status_ Separated', 'martial-status_ Widowed', 'occupation_ Adm-clerical', 'occupation_ Armed-Forces', 'occupation_ Craft-repair', 'occupation_ Exec-managerial', 'occupation_ Farming-fishing', 'occupa

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
X_train.shape

(26048, 100)

### Model Training

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 1s 782us/step - accuracy: 0.6648 - loss: 472.1396 - val_accuracy: 0.2365 - val_loss: 414.3333
Epoch 2/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 1s 675us/step - accuracy: 0.6777 - loss: 139.6965 - val_accuracy: 0.7942 - val_loss: 17.3010
Epoch 3/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 1s 697us/step - accuracy: 0.6854 - loss: 49.1168 - val_accuracy: 0.8019 - val_loss: 35.8321
Epoch 4/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 1s 680us/step - accuracy: 0.6751 - loss: 30.7395 - val_accuracy: 0.8054 - val_loss: 8.6935
Epoch 5/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 0s 662us/step - accuracy: 0.6782 - loss: 13.7026 - val_accuracy: 0.7927 - val_loss: 6.8598
Epoch 6/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 0s 661us/step - accuracy: 0.6873 - loss: 3.5886 - val_accuracy: 0.8054 - val_loss: 5.3231
Epoch 7/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 0s 660us/step - accuracy: 0.7015 - loss: 2.3460 - val_accuracy: 0.3466 - val_loss: 0.8639
Epoch 8/100
733/733 ━━━━━━━━━━━━━━━━━━━━ 0s 663us/step - accuracy: 0.730

In [10]:
loss, accuracy = model.evaluate(X_test, y_test)

204/204 ━━━━━━━━━━━━━━━━━━━━ 0s 353us/step - accuracy: 0.7715 - loss: 0.5325


In [11]:
y_pred = model.predict(X_test)

204/204 ━━━━━━━━━━━━━━━━━━━━ 0s 428us/step


## Model Card Generation

In [12]:
from patra_model_card.patra_model_card import ModelCard, AIModel, BiasAnalysis, ExplainabilityAnalysis

In [13]:
mc = ModelCard(
            name="UCI Adult Data Analysis model using Tensorflow",
            version="0.1",
            short_description="UCI Adult Data analysis using Tensorflow for demonstration of Patra Model Cards.",
            full_description="We have trained a ML model using the tensorflow framework to predict income for the UCI Adult Dataset. We leverage this data to run the Patra model cards to capture metadata about the model as well as fairness and explainability metrics.",
            keywords="uci adult, tensorflow, explainability, fairness, patra",
            author="Sachith Withana",
            input_type="Tabular",
            category="classification"
        )

mc.input_data = 'https://archive.ics.uci.edu/dataset/2/adult'
mc.output_data = 'https://github.iu.edu/d2i/dockerhub/tensorflow/adult_modelv01'

In [14]:
def remove_nulls(model_structure):
    if isinstance(model_structure, dict):
        return {k: remove_nulls(v) for k, v in model_structure.items() if v is not None and remove_nulls(v)}
    elif isinstance(model_structure, list):
        return [remove_nulls(v) for v in model_structure if v is not None and remove_nulls(v) != []]
    else:
        return model_structure

def get_model_structure(model_structure):
    json_structure = json.loads(model_structure)
    cleaned_structure = remove_nulls(json_structure)
    return cleaned_structure

In [15]:
ai_model = AIModel(
            name="Income prediction tensorflow model",
            version="0.1",
            description="Census classification problem using Tensorflow Neural Network using the UCI Adult Dataset",
            owner="Sachith Withana",
            location="https://github.iu.edu/d2i/uci_adult/tensorflow_model",
            license="BSD-3 Clause",
            framework="tensorflow",
            foundational_model="None",
            model_type="dnn",
            test_accuracy=accuracy,
            model_structure = get_model_structure(model.to_json())
        )


In [16]:
ai_model.add_metric("Test loss", loss)
ai_model.add_metric("Epochs", 100)
ai_model.add_metric("Batch Size", 32)
ai_model.add_metric("Optimizer", "Adam")
ai_model.add_metric("Learning Rate", 0.0001)
ai_model.add_metric("Input Shape", "(26048, 100)")

In [17]:
mc.ai_model = ai_model

In [18]:
mc.populate_requirements()

In [19]:
print(mc)

{
    "name": "UCI Adult Data Analysis model using Tensorflow",
    "version": "0.1",
    "short_description": "UCI Adult Data analysis using Tensorflow for demonstration of Patra Model Cards.",
    "full_description": "We have trained a ML model using the tensorflow framework to predict income for the UCI Adult Dataset. We leverage this data to run the Patra model cards to capture metadata about the model as well as fairness and explainability metrics.",
    "keywords": "uci adult, tensorflow, explainability, fairness, patra",
    "author": "Sachith Withana",
    "input_type": "Tabular",
    "category": "classification",
    "input_data": "https://archive.ics.uci.edu/dataset/2/adult",
    "output_data": "https://github.iu.edu/d2i/dockerhub/tensorflow/adult_modelv01",
    "ai_model": {
        "name": "Income prediction tensorflow model",
        "version": "0.1",
        "description": "Census classification problem using Tensorflow Neural Network using the UCI Adult Dataset",
       

In [20]:
y_pred = model.predict(X_test)
y_pred = (y_pred >= 0.5).flatten()
mc.populate_bias(X_test, y_test, y_pred, "gender", X_test[:, 58], model)

204/204 ━━━━━━━━━━━━━━━━━━━━ 0s 352us/step


In [21]:
x_columns = data.columns.tolist()
x_columns.remove('income')

mc.populate_xai(X_test[:10], x_columns, model, 10)

In [22]:
print(mc.bias_analysis)

{'demographic_parity_diff': 0.016893322677259265, 'equal_odds_difference': 0.015399962791175094}


In [23]:
print(mc.xai_analysis)

{'capital-gain': 0.08574622329324483, 'fnlwgt': 0.00804802388780647, 'age': 0.0001878592330548499, 'occupation_ Exec-managerial': 5.9379508925811665e-05, 'martial-status_ Married-civ-spouse': 3.860872652795555e-05, 'relationship_ Not-in-family': 3.367731968561703e-05, 'hours-per-week': 2.597770757145361e-05, 'education_ HS-grad': 2.5964784953329577e-05, 'martial-status_ Never-married': 1.6633578472667918e-05, 'workclass_ Private': 1.529919190539499e-05}


In [24]:
print(mc)

{
    "name": "UCI Adult Data Analysis model using Tensorflow",
    "version": "0.1",
    "short_description": "UCI Adult Data analysis using Tensorflow for demonstration of Patra Model Cards.",
    "full_description": "We have trained a ML model using the tensorflow framework to predict income for the UCI Adult Dataset. We leverage this data to run the Patra model cards to capture metadata about the model as well as fairness and explainability metrics.",
    "keywords": "uci adult, tensorflow, explainability, fairness, patra",
    "author": "Sachith Withana",
    "input_type": "Tabular",
    "category": "classification",
    "input_data": "https://archive.ics.uci.edu/dataset/2/adult",
    "output_data": "https://github.iu.edu/d2i/dockerhub/tensorflow/adult_modelv01",
    "ai_model": {
        "name": "Income prediction tensorflow model",
        "version": "0.1",
        "description": "Census classification problem using Tensorflow Neural Network using the UCI Adult Dataset",
       

In [25]:
mc.save("/Users/swithana/git/d2i/patra-toolkit/examples/model_cards/tesorflow_adult_nn_MC.json")

In [26]:
mc.validate()

True